In [2]:
pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 22.8 MB/s eta 0:00:00


In [3]:
import requests
from lxml import html
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke
import re

In [10]:
client = MongoClient('127.0.0.1', 27017)

In [11]:
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
           "Accept": "*/*"}
response = requests.get('https://lenta.ru/', headers=headers)
dom = html.fromstring(response.text)

In [12]:
items = dom.xpath('//div[@class = "b-yellow-box__wrap"]/div[@class = "item"]')

In [14]:
main_link = 'https://lenta.ru'
news_lenta = []
for item in items:
    item_date = {}
    news_sourse = 'https://lenta.ru/'
    news1 = item.xpath('.//a/text()')
    news1 = " ".join(news1[0].split())  #clean
    link1 = item.xpath('.//a/@href')
    link = main_link + link1[0]                 #full link
    date = '/'.join(link1[0].split('/')[2:5])

    item_date['source'] = news_sourse
    item_date['news'] = news1
    item_date['link'] = link
    item_date['date'] = date
    news_lenta.append(item_date)
pprint(news_lenta)

[]


In [ ]:
db = client['news_lenta']
l_news = db.l_news
for i in l_news.find():
    print(i)

In [17]:
import datetime

In [21]:
def mailru_news_agg():
  url = 'https://news.mail.ru/'
  response = requests.get(url, headers=headers)
  dom = html.fromstring(response.text)
  links = []
  links.extend(dom.xpath("//table/tr/td[1]/div[@class='daynews__item daynews__item_big']/a/@href"))
  links.extend(dom.xpath("//table/tr/td[2]/div[contains(@class, 'daynews__item')]/a/@href"))
  links.extend(dom.xpath("//div[@class='js-module']/ul/li[contains(@class,'list__item')]/a/@href"))
  mail_events = []
  for link in links:
    event = {}
    curr_link_response = requests.get(link, headers=headers)
    curr_dom = html.fromstring(curr_link_response.text)
    event['link'] = link
    try:
      event['date'] = datetime.datetime.strptime(curr_dom.xpath(".//h1[@class='hdr__inner']/../../../../div[contains(@class, 'breadcrumbs')]/span/span/span[contains(@class, 'js-ago')]/@datetime")[0][0:10], '%Y-%m-%d').date()
      event['source'] = curr_dom.xpath(".//h1[@class='hdr__inner']/../../../../div[contains(@class, 'breadcrumbs')]/span/span/a/span/text()")[0]
      event['name'] = curr_dom.xpath(".//h1[@class='hdr__inner']/text()")[0]
    except Exception:
      continue
    mail_events.append(event)
    return mail_events

In [ ]:
db = client['mail_news']
mail_news = db.mail_news
for i in mail_news.find():
    print(i)